In [4]:
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e1/58/9289c6a03116025cdb61461d99b2493daa4967a80b13755463d71a0affeb/geopy-2.4.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.3 kB ? eta -:--:--
     ------------------- ------------------ 20.5/40.3 kB 165.2 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/40.3 kB 163.8 kB/s eta 0:00:01
     -------------------------------------- 40.3/40.3 kB 174.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 k

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic

In [6]:
df = pd.read_csv('yelp_reviews.csv')
# df = df.drop(columns=['Unnamed: 0'])
df.head()

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,longitude,zip_codes,address,review_texts,review_ratings,review_times,chain_id_cosine,chain_id_jaccard,chain_id_levenshtein,chain_id_fuzzy
0,5744,pNbrxyrG5d5QwRsKbu0cXg,46-new-york,46,https://s3-media3.fl.yelpcdn.com/bphoto/PDgpYA...,False,https://www.yelp.com/biz/46-new-york?adjust_cr...,97,"[{'alias': 'desserts', 'title': 'Desserts'}]",4.5,...,-73.998644,10013,46 Mott St,"[""Cash only for transactions under $5. \n\nAma...","[5, 3, 4]","[""2023-05-14 12:05:28"", ""2023-08-23 20:49:11"",...",NaN,NaN,NaN,NaN
1,13902,8Oo2AtQEPDfxIOnA8wfXoQ,886-new-york,886,https://s3-media3.fl.yelpcdn.com/bphoto/N-V-Dl...,False,https://www.yelp.com/biz/886-new-york?adjust_c...,693,"[{'alias': 'taiwanese', 'title': 'Taiwanese'},...",4.0,...,-73.988730,10003,26 Saint Marks Pl,"[""Love that this place kind of mimic the Asian...","[5, 4, 1]","[""2023-10-15 19:30:27"", ""2023-11-11 17:09:46"",...",1.0,1.0,1.0,1.0
2,95,fNKYBpQJivICm0ORb3nb_Q,8282-new-york-2,8282,https://s3-media2.fl.yelpcdn.com/bphoto/jMQtrS...,False,https://www.yelp.com/biz/8282-new-york-2?adjus...,178,"[{'alias': 'korean', 'title': 'Korean'}]",4.5,...,-73.988750,10002,84 Stanton St,"[""This is a small Korean fusion tapas restaura...","[5, 5, 5]","[""2023-11-08 19:11:14"", ""2023-10-26 22:42:57"",...",2.0,2.0,2.0,2.0
3,22197,Xu1jidsfdN0Wb3RMII3abw,apr57-new-york-2,Apr-57,https://s3-media1.fl.yelpcdn.com/bphoto/osmh-5...,False,https://www.yelp.com/biz/apr57-new-york-2?adju...,28,"[{'alias': 'jewelry', 'title': 'Jewelry'}, {'a...",2.5,...,-73.980528,10019,200 W 57th St,NaN,NaN,NaN,3.0,3.0,3.0,3.0
4,11459,cuqT0Venx12D4bil06J1UA,purnima-halal-brooklyn,Purnima Halal,NaN,False,https://www.yelp.com/biz/purnima-halal-brookly...,1,"[{'alias': 'grocery', 'title': 'Grocery'}]",5.0,...,-73.951040,11216,1257 Fulton St,"[""Wide selection, all clean and new. Welcoming...",[5],"[""2018-03-20 02:10:06""]",4.0,4.0,4.0,4.0


In [8]:
# Create a function to calculate Haversine distance
def haversine_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

# Create an empty DataFrame to store average distances
avg_distances_df = pd.DataFrame(columns=['chain_id_fuzzy', 'name', 'average_dist'])

# Iterate through each chain_id_fuzzy
for chain_id, group in df.groupby('chain_id_fuzzy'):
    # Get unique business names within the chain
    unique_names = group['name'].unique()

    # Calculate pairwise distances between outlets
    distances = []
    for name in unique_names:
        outlets = group[group['name'] == name][['id', 'latitude', 'longitude']]
        for i, outlet1 in outlets.iterrows():
            for j, outlet2 in outlets.iterrows():
                if i < j:
                    distance = haversine_distance((outlet1['latitude'], outlet1['longitude']),
                                                  (outlet2['latitude'], outlet2['longitude']))
                    distances.append(distance)

    # Calculate average distance for the chain
    if distances:
        average_dist = sum(distances) / len(distances)
    else:
        average_dist = 0

    # Append to the new DataFrame
    avg_distances_df = pd.concat([avg_distances_df,
                                  pd.DataFrame({'chain_id_fuzzy': [chain_id],
                                                'name': [name],
                                                'average_dist': [average_dist]})],
                                 ignore_index=True)

# Display the resulting DataFrame
# avg_distances_df.sort_values(by="average_dist", inplace=True)
avg_distances_df

,chain_id_fuzzy,name,average_dist
0,1.0,886,0
1,2.0,8282,0
2,3.0,Apr-57,0
3,4.0,Purnima Halal,0
4,5.0,#RMHQ,0
...,...,...,...
18830,18831.0,Zutto Japanese American Pub,0
18831,18832.0,Zuzu's Petals,0
18832,18833.0,Zyara Restaurant,0
18833,18834.0,Zyoga Space,0


In [10]:
avg_distances_df.to_csv("average_dist_businesses.csv")